In [21]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import streamlit as st


from sklearn import datasets # sklearn comes with some toy datasets to practice
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from PIL import Image
from IPython.display import display
from fuzzywuzzy import fuzz, process


%matplotlib inline

C:\Users\espan\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Training our KMeans

In [3]:
df= pd.read_csv("all_recipes_FV.csv")

In [4]:
X = df[["Protein(g)","Protein Calorie","Carbs(g)","Carbs Calorie", "Fat(g)", "Fat Calorie", "Total Calories"]]

In [5]:
scaler = StandardScaler()

scaler.fit(X)

X_scaled = scaler.transform(X)  # we are getting an array

# we need to get our dataframe

X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)

In [6]:
# KMeans 01

kmeans = KMeans(n_clusters=10, random_state=1234)
kmeans.fit(X_scaled_df)

kmeans.inertia_

12691.671168266412

In [7]:
# KMeans Lebels
labels = kmeans.labels_

In [8]:
#Clusters
clusters = kmeans.predict(X_scaled_df)

In [10]:
# Adding Cluster in X
X["cluster"] = clusters

C:\Users\espan\AppData\Local\Temp\ipykernel_28232\431869542.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["cluster"] = clusters


In [80]:
# Final Data Frame
df_final = pd.merge(X, df[['Recipe_name', "Cuisine_type","Diet_type"]], left_index=True, right_index=True)

In [81]:
df_final

,Protein(g),Protein Calorie,Carbs(g),Carbs Calorie,Fat(g),Fat Calorie,Total Calories,cluster,Recipe_name,Cuisine_type,Diet_type
0,5.22,20.88,1.29,5.16,3.20,12.80,38.84,0,Bone Broth From 'Nom Nom Paleo',american,paleo
1,181.55,726.20,28.62,114.48,146.14,584.56,1425.24,6,"Paleo Effect Asian-Glazed Pork Sides, A Sweet ...",south east asian,paleo
2,30.91,123.64,302.59,1210.36,96.76,387.04,1721.04,9,Paleo Pumpkin Pie,american,paleo
3,9.62,38.48,75.78,303.12,59.89,239.56,581.16,0,Strawberry Guacamole recipes,mexican,paleo
4,39.84,159.36,54.08,216.32,71.55,286.20,661.88,0,"Asian Cauliflower Fried ""Rice"" From 'Nom Nom P...",chinese,paleo
...,...,...,...,...,...,...,...,...,...,...,...
7801,85.20,340.80,288.14,1152.56,137.15,548.60,2041.96,9,Brown Butter-Sunchoke Soup With Brussels Sprou...,italian,dash
7802,141.98,567.92,123.18,492.72,323.50,1294.00,2354.64,2,Make-Your-Own-Salad,american,dash
7803,0.01,0.04,0.39,1.56,0.00,0.00,1.60,0,Luis Buñuel Dry Martini,world,dash
7804,155.38,621.52,239.88,959.52,260.84,1043.36,2624.40,2,Cornflake Semi-Fried Chicken Tenders,american,dash


In [82]:
df_final.columns

Index(['Protein(g)', 'Protein Calorie', 'Carbs(g)', 'Carbs Calorie', 'Fat(g)',
       'Fat Calorie', 'Total Calories', 'cluster', 'Recipe_name',
       'Cuisine_type', 'Diet_type'],
      dtype='object')

###  User Inputs

## Total Calories per  MEAL🥗✅ 

In [22]:
def get_non_empty_float(prompt):
    while True:
        try:
            value = float(input(prompt))
            if value <= 0:
                print("Please enter a positive non-zero value.")
            else:
                return value
        except ValueError:
            print("Invalid input. Please enter a valid number.")

In [23]:
def get_gender():
    while True:
        gender = input("Gender (male/female): ").lower()
        if gender in ['male', 'female']:
            return gender
        else:
            print("Invalid gender. Please enter 'male' or 'female'.")

In [24]:
def get_activity_level():
    
    # Define a list of correct spellings or keywords:
    # correct_spellings = ["sedentary", "lightly active", "moderately active", "very active"]

    activity_levels = ['sedentary', 'lightly active', 'moderately active', 'very active', 'extra active']
    

    
    
    while True:
        nivel_activity = input("Activity Level (sedentary/lightly active/moderately active/very active/extra active): ").lower()
        
        # closest match
        closest_match, similarity_score = process.extractOne(nivel_activity, activity_levels)
        
        if nivel_activity in activity_levels:
            return nivel_activity
        else:
            print("Invalid activity level. Please choose from the provided options.")

In [25]:
def calculate_bmr(weight, height, age, gender):
    # BMR calculation using the Harris-Benedict equation
    if gender == "male":
        bmr = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
    else:
        bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)
    return bmr

In [87]:
def calculate_macronutrient_distribution(TDEE):
    
    # List to store user inputs
    user_data = []  
    
    # Macronutrient Ratios
    protein_ratio = 0.3
    fat_ratio = 0.3
    carbohydrate_ratio = 0.4

    # Caloric content per gram of each macronutrient
    protein_calories_per_gram = 4
    fat_calories_per_gram = 9
    carbohydrate_calories_per_gram = 4

    # Calculate the grams of each macronutrient
    protein_grams = (protein_ratio * TDEE) / protein_calories_per_gram
    fat_grams = (fat_ratio * TDEE) / fat_calories_per_gram
    carbohydrate_grams = (carbohydrate_ratio * TDEE) / carbohydrate_calories_per_gram

    # Calculate the calories contributed by each macronutrient
    protein_calories = protein_grams * protein_calories_per_gram
    fat_calories = fat_grams * fat_calories_per_gram
    carbohydrate_calories = carbohydrate_grams * carbohydrate_calories_per_gram
    
    #Total Calories
    total_calories = carbohydrate_calories + fat_calories + protein_calories
    
    user_data.append(
            {
        "Protein(g)": protein_grams,
        "Protein Calorie": protein_calories,
        "Carbs(g)": carbohydrate_grams,
        "Carbs Calorie": carbohydrate_calories,
        "Fat(g)": fat_grams,
        "Fat Calorie": fat_calories,
        "Total Calories": total_calories
    })
        
    user_inputs_df = pd.DataFrame(user_data)
    

    return {
        "Protein(g)": protein_grams,
        "Protein Calorie": protein_calories,
        "Carbs(g)": carbohydrate_grams,
        "Carbs Calorie": carbohydrate_calories,
        "Fat(g)": fat_grams,
        "Fat Calorie": fat_calories,
        "Total Calories": total_calories
    }


def TDEE_calculator_per_meal():
    print("Welcome to the TDEE Calculator!")
    print("Please enter the following details:")

    weight = get_non_empty_float("Weight (kg): ")
    height = get_non_empty_float("Height (cm): ")
    age = get_non_empty_float("Age (years): ")
    gender = get_gender()
    nivel_activity = get_activity_level()
    
    # Basal Metabolic Rate (BMR)

    bmr = calculate_bmr(weight, height, age, gender)

    activity_factors = {
        "sedentary": 1.2,
        "lightly active": 1.375,
        "moderately active": 1.55,
        "very active": 1.725,
        "extra active": 1.9
    }
    
    ##### per meal
    
    # Create a list to hold the data for the DataFrame
    user_inputs_parameters = []
    
    if nivel_activity in activity_factors:
        TDEE = (bmr * activity_factors[nivel_activity])/3
        print(f"Your Total Daily Energy Expenditure (TDEE) is: {TDEE:.2f} calories per day.")

        # Calculate and display the macronutrient distribution
        macronutrient_distribution = calculate_macronutrient_distribution(TDEE)
        
        print("\nMacronutrient Distribution:")
        
        for macronutrients, value in macronutrient_distribution.items():
            
            
            #print(f"{macronutrients}: {value:.2f}")
            
            # Append the nutrient and value to the data list as a tuple
            user_inputs_parameters.append((macronutrients, value))
            
            # Create a DataFrame from the data list
            df_user_inputs = pd.DataFrame(user_inputs_parameters, columns=["Macronutrients", "Value"])
            
            #print("\n User Macronutrients Distributions DataFrame:")
            #print(df_user_inputs)
            
            
            #######################################################################
            # Transpose the DataFrame
            df_user_inputs = df_user_inputs_parameter.transpose()


            # Changing the columns names

            custom_column_names = {
                  "0": "Protein(g)",
                  "1": "Protein Calorie",
                  "2": "Carbs(g)",
                  "3": "Carbs Calorie",
                  "4": "Fat(g)",
                  "5": "Fat Calorie",
                  "6": "Total Calories"}

            df_user_inputs.columns = custom_column_names.values()


            # Dropping the columns "Macronutrients"

            df_user_inputs = df_user_inputs.drop(index="Macronutrients")

            # Resetting the index
            df_user_inputs = df_user_inputs.reset_index(drop=True)
            
            ######################################################################
            
            
      

    else:
        print("Invalid activity level. Please choose from the provided options.")
        
    return df_user_inputs 

In [88]:
TDEE_calculator_per_meal()

Welcome to the TDEE Calculator!
Please enter the following details:
Weight (kg): 68
Height (cm): 177
Age (years): 31
Gender (male/female): female
Activity Level (sedentary/lightly active/moderately active/very active/extra active): moderately active
Your Total Daily Energy Expenditure (TDEE) is: 770.09 calories per day.

Macronutrient Distribution:


,Protein(g),Protein Calorie,Carbs(g),Carbs Calorie,Fat(g),Fat Calorie,Total Calories
0,57.757069,231.028275,77.009425,308.0377,25.669808,231.028275,770.09425


In [89]:
df_user_inputs_parameter = TDEE_calculator_per_meal()

Welcome to the TDEE Calculator!
Please enter the following details:
Weight (kg): 68
Height (cm): 177
Age (years): 31
Gender (male/female): female
Activity Level (sedentary/lightly active/moderately active/very active/extra active): moderately active
Your Total Daily Energy Expenditure (TDEE) is: 770.09 calories per day.

Macronutrient Distribution:


In [90]:
df_user_inputs_parameter

,Protein(g),Protein Calorie,Carbs(g),Carbs Calorie,Fat(g),Fat Calorie,Total Calories
0,57.757069,231.028275,77.009425,308.0377,25.669808,231.028275,770.09425


### Recommending a Recipe to our User

In [102]:
def recipe_recommendation():
    
    
    df_user_inputs_parameter = TDEE_calculator_per_meal()


     # Scaling the user inputs DataFrame

    scaler = StandardScaler()

    scaler.fit(df_user_inputs_parameter)
    scaler.transform(df_user_inputs_parameter)
    user_inputs_scaled = scaler.transform(df_user_inputs_parameter)

    # Apply KMeans prediction to user_inputs_scaled
    user_cluster = kmeans.predict(user_inputs_scaled)

    # Filter recommended recipe based on predicted cluster

    recommended_recipe_from_cluster = df_final[df_final['cluster'] == user_cluster[0]]

    # Select a random recipe from recommended recipes
    recommended_recipe = recommended_recipe_from_cluster.sample(n=1)
    
    return recommended_recipe


In [ ]:
def calculate_macronutrient_distribution(TDEE):
    
    # List to store user inputs
    user_data = []  
    
    # Macronutrient Ratios
    protein_ratio = 0.3
    fat_ratio = 0.3
    carbohydrate_ratio = 0.4

    # Caloric content per gram of each macronutrient
    protein_calories_per_gram = 4
    fat_calories_per_gram = 9
    carbohydrate_calories_per_gram = 4

    # Calculate the grams of each macronutrient
    protein_grams = (protein_ratio * TDEE) / protein_calories_per_gram
    fat_grams = (fat_ratio * TDEE) / fat_calories_per_gram
    carbohydrate_grams = (carbohydrate_ratio * TDEE) / carbohydrate_calories_per_gram

    # Calculate the calories contributed by each macronutrient
    protein_calories = protein_grams * protein_calories_per_gram
    fat_calories = fat_grams * fat_calories_per_gram
    carbohydrate_calories = carbohydrate_grams * carbohydrate_calories_per_gram
    
    #Total Calories
    total_calories = carbohydrate_calories + fat_calories + protein_calories
    
    user_data.append(
            {
        "Protein(g)": protein_grams,
        "Protein Calorie": protein_calories,
        "Carbs(g)": carbohydrate_grams,
        "Carbs Calorie": carbohydrate_calories,
        "Fat(g)": fat_grams,
        "Fat Calorie": fat_calories,
        "Total Calories": total_calories
    })
        
    user_inputs_df = pd.DataFrame(user_data)
    

    return {
        "Protein(g)": protein_grams,
        "Protein Calorie": protein_calories,
        "Carbs(g)": carbohydrate_grams,
        "Carbs Calorie": carbohydrate_calories,
        "Fat(g)": fat_grams,
        "Fat Calorie": fat_calories,
        "Total Calories": total_calories
    }


def TDEE_calculator_per_meal():
    print("Welcome to the TDEE Calculator!")
    print("Please enter the following details:")

    weight = get_non_empty_float("Weight (kg): ")
    height = get_non_empty_float("Height (cm): ")
    age = get_non_empty_float("Age (years): ")
    gender = get_gender()
    nivel_activity = get_activity_level()
    
    # Basal Metabolic Rate (BMR)

    bmr = calculate_bmr(weight, height, age, gender)

    activity_factors = {
        "sedentary": 1.2,
        "lightly active": 1.375,
        "moderately active": 1.55,
        "very active": 1.725,
        "extra active": 1.9
    }
    
    ##### per meal
    
    # Create a list to hold the data for the DataFrame
    user_inputs_parameters = []
    
    if nivel_activity in activity_factors:
        TDEE = (bmr * activity_factors[nivel_activity])/3
        print(f"Your Total Daily Energy Expenditure (TDEE) is: {TDEE:.2f} calories per day.")

        # Calculate and display the macronutrient distribution
        macronutrient_distribution = calculate_macronutrient_distribution(TDEE)
        
        print("\nMacronutrient Distribution:")
        
        for macronutrients, value in macronutrient_distribution.items():
            
            
            #print(f"{macronutrients}: {value:.2f}")
            
            # Append the nutrient and value to the data list as a tuple
            user_inputs_parameters.append((macronutrients, value))
            
            # Create a DataFrame from the data list
            df_user_inputs = pd.DataFrame(user_inputs_parameters, columns=["Macronutrients", "Value"])
            
            #print("\n User Macronutrients Distributions DataFrame:")
            #print(df_user_inputs)
            
            
            #######################################################################
            # Transpose the DataFrame
            df_user_inputs = df_user_inputs_parameter.transpose()


            # Changing the columns names

            custom_column_names = {
                  "0": "Protein(g)",
                  "1": "Protein Calorie",
                  "2": "Carbs(g)",
                  "3": "Carbs Calorie",
                  "4": "Fat(g)",
                  "5": "Fat Calorie",
                  "6": "Total Calories"}

            df_user_inputs.columns = custom_column_names.values()


            # Dropping the columns "Macronutrients"

            df_user_inputs = df_user_inputs.drop(index="Macronutrients")

            # Resetting the index
            df_user_inputs = df_user_inputs.reset_index(drop=True)
            
            ######################################################################
            
             df_user_inputs_parameter = TDEE_calculator_per_meal()


            # Scaling the user inputs DataFrame

            scaler = StandardScaler()

            scaler.fit(df_user_inputs_parameter)
            scaler.transform(df_user_inputs_parameter)
            user_inputs_scaled = scaler.transform(df_user_inputs_parameter)

    # Apply KMeans prediction to user_inputs_scaled
    user_cluster = kmeans.predict(user_inputs_scaled)

    # Filter recommended recipe based on predicted cluster

    recommended_recipe_from_cluster = df_final[df_final['cluster'] == user_cluster[0]]

    # Select a random recipe from recommended recipes
    recommended_recipe = recommended_recipe_from_cluster.sample(n=1)
      

    else:
        print("Invalid activity level. Please choose from the provided options.")
        
    return df_user_inputs 

### Draft

In [ ]:
# Scaling the user inputs DataFrame

scaler = StandardScaler()

scaler.fit(df_user_inputs_parameter)
scaler.transform(df_user_inputs_parameter)
user_inputs_scaled = scaler.transform(df_user_inputs_parameter)

# Apply KMeans prediction to user_inputs_scaled
user_cluster = kmeans.predict(user_inputs_scaled)

# Filter recommended recipe based on predicted cluster

recommended_recipe_from_cluster = df_final[df_final['cluster'] == user_cluster[0]]

# Select a random recipe from recommended recipes
recommended_recipe = recommended_recipe_from_cluster.sample(n=1)